Avances hasta ahora: 

1. Empezamos con una poblacion base de funciones y otra de operaciones
2. Se crea la generación 0 combinando estas dos poblaciones bases
3. Se seleccionan individuos de la población 0
4. Estos se combinan de a pares, de forma tal que al final de la combinación queda el mismo numero de individuos de la poblacion 0
5. Se escogen aleatoriamente algunos para mutarlos
6. Se crea generación nueva

Errores: 
1. No sé porqué a veces el chi me da un numero imaginario
2. Creo que  en el nueva generación debería conservar individuos de la antigua, y que dentro de esta no solo esten los productos de la combinación/mutación



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import symbols, sympify, lambdify, compose, I, pi
from sympy.abc import x

In [2]:
# Extracción de datos

columna1 = []
columna2 = []
columna3 = []

with open('c:/Users/ximen/Desktop/Practicas DFI/hub_data_3.txt', 'r') as archivo:
    # Lee cada línea del archivo
    for linea in archivo: #cada linea es una fila del texto
        elementos = linea.split() # divide la línea en sus elementos utilizando espacios como delimitadores
        #print(elementos)
        columna1.append(elementos[0]) # Agrega cada elemento a su respectiva columna
        columna2.append(elementos[1])
        columna3.append(elementos[2])

z = np.zeros(len(columna1))
for i in range(len(columna1)):
  z[i] = float(columna1[i])

H = np.zeros(len(columna2))
for i in range(len(columna2)):
  H[i] = float(columna2[i])

desv = np.zeros(len(columna3))
for i in range(len(columna3)):
  desv[i] = float(columna3[i])

In [3]:
# POBLACION BASE

x = symbols('x')

poblacion_base = ['exp(x)', 'log(x)', 'x', 'x**2']

operaciones = ['*', '**', '/', '+', '-', 'comp'] # Agregar 'comp' para representar la combinacion

In [4]:
def Creador(num_poblacion_inicial, poblacion_base, operaciones):
    '''
    Creador recibe: 
    1. El numero de individuos que se desea que tenga la poblacion inicial
    2. Una lista con funciones base cuyas combinaciones crearan la poblacion inicial
    3. una lista con operaciones que actuarán sobre las funciones al momento de combinarlas
    '''
    poblacion_inicial= []
    for n in range(num_poblacion_inicial):
        expresion1 = random.choice(poblacion_base)
        expresion2 = random.choice(poblacion_base)
        operacion = random.choice(operaciones)
        if operacion == 'comp':
            # Se pasan a funciones sympy
            x = symbols('x')
            expresiones = [expresion1, expresion2]
            expresiones_sympy = [sympify(exp) for exp in expresiones] 
            f1 = expresiones_sympy[0]
            f2 = expresiones_sympy[1]
            # Se componen expresiones
            composicion = f1.subs(x, f2)
            poblacion_inicial.append(str(composicion))
        else: 
            expresion = f"{expresion1} {operacion} {expresion2}"
            poblacion_inicial.append(expresion)
    return poblacion_inicial

In [5]:
# Comprobacion de que creador esta bien

poblacion_inicial = Creador(10, poblacion_base, operaciones)

print(poblacion_inicial)

['exp(x) + x', 'exp(x) + exp(x)', 'exp(x) - x**2', 'exp(x) + x', 'log(x**2)', 'exp(x) / x', 'log(x)**2', 'x ** x', 'x + x', 'x ** x**2']


In [6]:
def Seleccionador(poblacion_inicial, valores_observados, porcentaje_seleccion):
    '''
    Seleccionador recibe: 
    1. La poblacion inicial de funciones
    2. Los valores observados (que se esperan ajustar a una función)
    3. El porcentaje que se quiere de selección
    '''
    valores_x = valores_observados.tolist()
    Chi = np.zeros(len(poblacion_inicial))
    x = symbols('x')
    expresiones_sympy = [sympify(exp) for exp in poblacion_inicial] 
    
    for n , funcion in enumerate(expresiones_sympy):
        f = [funcion.subs(x, valor) for valor in valores_x]
        diferencia = [resultado - valor for resultado, valor in zip(z, f)]
        chi = np.sum((diferencia/desv)**2)
        chi_real, chi_imaginario = expresion = chi.as_real_imag()
        Chi[n] = chi_real

    seleccion = np.percentile(Chi, porcentaje_seleccion)
    seleccionados = Chi[Chi <= seleccion]
    indices = np.where(Chi <= seleccion)[0]
    funciones_seleccionadas = [poblacion_inicial[i] for i in indices]
    return funciones_seleccionadas

In [7]:
# Comprobacion de que Seleccionador esta bien

funciones_seleccionadas = Seleccionador(poblacion_inicial, z, 50)
print(funciones_seleccionadas)

['exp(x) - x**2', 'log(x**2)', 'log(x)**2', 'x ** x', 'x + x']


In [8]:
def Combinador(funciones_seleccionadas, operaciones): 
    
    combinados = []
    num_combinados = len(funciones_seleccionadas)

    for n in range(num_combinados):
        indice = random.randint(0, len(funciones_seleccionadas)-1)
        expresion1 = funciones_seleccionadas[n]
        expresion2 = funciones_seleccionadas[indice]
        operacion = random.choice(operaciones)
        if operacion == 'comp':
            # Se pasan a funciones sympy
            x = symbols('x')
            expresiones = [expresion1, expresion2]
            #print(expresiones)
            expresiones_sympy = [sympify(exp) for exp in expresiones] 
            f1 = expresiones_sympy[0]
            #print(f1)
            f2 = expresiones_sympy[1]
            #print(f2)
            # Se componen expresiones
            composicion = f1.subs(x, f2)
            #print(composicion)
            combinados.append(str(composicion))
        else: 
            expresion = f"{expresion1} {operacion} {expresion2}"
            combinados.append(expresion)

    return combinados

In [9]:
# Comprobacion de que Combinador esta bien

funciones_combinadas = Combinador(funciones_seleccionadas, operaciones)
print(funciones_combinadas)
print(len(funciones_combinadas))

['exp(x) - x**2 / log(x**2)', 'log(x**2) / log(x**2)', 'log(x)**2 ** exp(x) - x**2', 'x ** x ** x + x', 'x + x - log(x)**2']
5


In [10]:
def Mutador(porcentaje_mutacion, funciones_combinadas, poblacion_base, operaciones):
    num_mutacion = int(len(funciones_combinadas)*(porcentaje_mutacion/100))
    
    mutaciones = []
    funciones_a_mutar = []

    for n in range(0, num_mutacion): # range no cuenta el ultimo elemento
        if n == 0:
            expresion1 = random.choice(funciones_combinadas)
            funciones_a_mutar.append(expresion1)
        else:
            expresion1 = random.choice(funciones_combinadas)
            while expresion1 in funciones_a_mutar:
                expresion1 = random.choice(funciones_combinadas)
            funciones_a_mutar.append(expresion1)

        indices = [i for i, elem in enumerate(funciones_combinadas) if elem in funciones_a_mutar]

        expresion2 = random.choice(poblacion_base)

        operacion = random.choice(operaciones)

        if operacion == 'comp':
            x = symbols('x')
            expresiones = [expresion1, expresion2]
            expresiones_sympy = [sympify(exp) for exp in expresiones] 
            f1 = expresiones_sympy[0]
            f2 = expresiones_sympy[1]
            # Se componen expresiones
            composicion = f1.subs(x, f2)
            mutaciones.append(str(composicion))
        else: 
            expresion = f"({expresion1}) {operacion} {expresion2}"
            mutaciones.append(expresion)

    return mutaciones, indices

In [11]:
# Comprobacion de que Mutador esta bien

funciones_mutadas, indices = Mutador(50, funciones_combinadas, poblacion_base, operaciones)

print(funciones_mutadas)
print(indices)

['1', '(x ** x ** x + x) - exp(x)']
[1, 3]


In [12]:
def Nueva_Generacion(funciones_combinadas, mutaciones, indices):
    nueva_generacion = []

    i = 0
    for n in range(0, len(funciones_combinadas)):
        if n in indices:
            nueva_generacion.append(mutaciones[i])
            i += 1
        else:
            nueva_generacion.append(funciones_combinadas[n])
    
    return nueva_generacion

In [13]:
# Comprobar si Nueva_Generacion está bien

A = Nueva_Generacion(funciones_combinadas, funciones_mutadas, indices)

print(A)
print(len(A))

['exp(x) - x**2 / log(x**2)', '1', 'log(x)**2 ** exp(x) - x**2', '(x ** x ** x + x) - exp(x)', 'x + x - log(x)**2']
5
